In [1]:
import pandas as pd
import seaborn as sns
import pickle
import matplotlib.pyplot as plt

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1747998297301, experiment_id='1', last_update_time=1747998297301, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    #Standart deviation of the trips duration in January
    print('Standart deviation: ', df.duration.std())
    
    # Fraction of the records between 1 and 60 minutes
    print('Fraction: ', ((df.duration >= 1) & (df.duration <= 60)).mean())
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    return df

In [5]:
df_train = read_dataframe('data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('data/yellow_tripdata_2023-02.parquet')

Standart deviation:  42.59435124195458
Fraction:  0.9812202822125979
Standart deviation:  42.84210176105113
Fraction:  0.9800944077722545


In [6]:
# df_jan = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet")
# df_feb = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet")
# df_march = read_dataframe("https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet")

In [6]:
categorical = ['PULocationID','DOLocationID']
df_train[categorical] = df_train[categorical].astype(str)

categorical = ['PULocationID','DOLocationID']
df_val[categorical] = df_val[categorical].astype(str)

In [ ]:
dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

: 

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

print('Dimensionality: ', X_train.shape)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_train = lr.predict(X_train)
y_pred_val = lr.predict(X_val)

print('RMSE on train: ', mean_squared_error(y_train, y_pred_train)**0.5)
print('RMSE on val: ', mean_squared_error(y_val, y_pred_val)**0.5)

RMSE on train:  7.6492624397080675
RMSE on val:  7.81181211389241


In [10]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
sns.distplot(y_pred_train, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [ ]:
sns.distplot(y_pred_val, label='prediction')
sns.distplot(y_val, label='actual')

plt.legend()

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("ML eng", "Ulya")
    
    mlflow.log_param("train-data-path", 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
    mlflow.log_param("val-data-path", "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")
    
    alpha = 0.1
    
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred_val)**0.5
    mlflow.log_param("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

ValueError: Found input variables with inconsistent numbers of samples: [3009173, 2855951]

In [13]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
import numpy as np

In [16]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective (params):

    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50
        )

        y_pred = booster.predict(valid)
        rmse=np.sqrt(mean_squared_error(y_val, y_pred))
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

: 

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn = objective,
    space = search_space,
    algo = tpe.suggest,
    max_evals = 50,
    trials = Trials()
)

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learing_rate': 0.09585355369315604,
        'max_depth': 30, 
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_lambda': 0.18060244040060163,
        'reg_alpha': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params = best_params,
        dtrain = train,
        num_boost_round=10,
        evals=[(valid, "validation")],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric("rmse", rmse)

    with open ("models/preprocessor.b", "w") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")
    


/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:23:32] WARNING: /workspace/src/objective/regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/opt/conda/envs/exp-tracking-env/lib/python3.9/site-packages/xgboost/core.py:158: UserWarning: [11:23:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "learing_rate" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:8.89990
[1]	validation-rmse:8.22499
[2]	validation-rmse:7.75632
[3]	validation-rmse:7.53729
[4]	validation-rmse:7.35711
[5]	validation-rmse:7.24226
[6]	validation-rmse:7.15670
[7]	validation-rmse:7.10534
[8]	validation-rmse:7.05716
[9]	validation-rmse:6.76252
[10]	validation-rmse:6.71882
[11]	validation-rmse:6.67004
[12]	validation-rmse:6.64179
[13]	validation-rmse:6.61355
[14]	validation-rmse:6.41972
[15]	validation-rmse:6.40315
[16]	validation-rmse:6.38974
[17]	validation-rmse:6.35758
[18]	validation-rmse:6.26077
[19]	validation-rmse:6.25000
[20]	validation-rmse:6.14748
[21]	validation-rmse:6.13215
[22]	validation-rmse:6.12343
[23]	validation-rmse:6.11454
[24]	validation-rmse:6.09628
[25]	validation-rmse:6.08828
[26]	validation-rmse:6.07869
[27]	validation-rmse:6.07369
[28]	validation-rmse:5.96712
[29]	validation-rmse:5.95606
[30]	validation-rmse:5.94287
[31]	validation-rmse:5.93576
[32]	validation-rmse:5.92554
[33]	validation-rmse:5.91712
[34]	validation-rmse:5.9

: 